# Basic RAG with Model-graded Eval

In this example we'll build a simple RAG application on Volume 7 of History of the United States of America, 
and evaluate it across 
* **relevance** -- does the answer make sense in context of the original question?, 
* **faithfulness** -- is the final answer faithful to the data that we fed into the LLM?
* **coherence** -- is the answer consistent and easy to understand?

We'll use AIConfig to manage and iterate on all our prompts, both for the generation step of the RAG pipeline, as well as its evaluation.

## Install dependencies

Create .env file containing the following line:
`OPENAI_API_KEY=<your key here>`
> You can get your key from https://platform.openai.com/api-keys 


In [1]:
!pip install python-aiconfig==1.1.20
!pip install chromadb

import dotenv
dotenv.load_dotenv()

  Using cached importlib_metadata-5.2.0-py3-none-any.whl (21 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.11.0
    Uninstalling importlib-metadata-6.11.0:
      Successfully uninstalled importlib-metadata-6.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-api 1.22.0 requires importlib-metadata<7.0,>=6.0, but you have importlib-metadata 5.2.0 which is incompatible.
  Using cached importlib_metadata-6.11.0-py3-none-any.whl.metadata (4.9 kB)
Using cached importlib_metadata-6.11.0-py3-none-any.whl (23 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.2.0
    Uninstalling importlib-metadata-5.2.0:
      Successfully uninstalled importlib-metadata-5.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that 

True

In [2]:
import argparse
import asyncio
import os
import sys
from aiconfig import AIConfigRuntime
import chromadb
from glob import glob

/opt/homebrew/Caskroom/miniconda/base/envs/rag-demo-1/lib/python3.10/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_parsers" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/rag-demo-1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Download the raw data
Fetch Volume 7 of the History of the United States of America (our raw unstructured dataset)

In [3]:
!mkdir -p data/books/
!wget https://www.gutenberg.org/cache/epub/72846/pg72846.txt -O data/books/pg72846.txt

--2024-02-08 16:38:55--  https://www.gutenberg.org/cache/epub/72846/pg72846.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 651483 (636K) [text/plain]
Saving to: ‘data/books/pg72846.txt’

data/books/pg72846. 100%[===================>] 636.21K  --.-KB/s    in 0.1s    

2024-02-08 16:38:56 (4.30 MB/s) - ‘data/books/pg72846.txt’ saved [651483/651483]



In [4]:
!head data/books/pg72846.txt

The Project Gutenberg eBook of History of the United States of America, Volume 7 (of 9)
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.



In [5]:
collection_name="us_history_volume_7"
chromadb_path="chroma_2.db"

## RAG Data Ingestion & Indexing
Chunk the data and ingest it into a Chroma DB collection.

> We use a very naive text splitting strategy with fixed-size chunks. For a production environment, this step will be critical to optimize.

**Note:** You can also run this as a CLI script using the command 
```
!python rag.py ingest `data/books/` --chroma-collection-name us_history_volume_7
```

In [6]:
def chunk_markdown(text, chunk_size=1000):
    chunks = []
    for i in range(0, len(text), chunk_size):
        yield text[i : i + chunk_size]
    return chunks

In [7]:
async def run_ingest(directory, collection_name):
    chroma_client = chromadb.PersistentClient(path=chromadb_path)
    collection = chroma_client.create_collection(name=collection_name)

    for i, filename in enumerate(glob(f"{directory}/**/*", recursive=True)):
        print("Ingesting:", i, filename)
        documents = []
        metadatas = []
        ids = []

        with open(filename, "r") as f:
            data = f.read()
            for j, chunk in enumerate(chunk_markdown(data)):
                documents.append(chunk)
                metadatas.append({"source": filename})
                ids.append(f"doc_{i}_chunk{j}")

        collection.add(documents=documents, metadatas=metadatas, ids=ids)

In [8]:
try:
    await run_ingest(directory="data/books", collection_name=collection_name)
except Exception as e:
    print(f"Ingest failed: {e}.\nIf the collection exists already, this is fine.")

Ingest failed: Collection us_history_volume_7 already exists.
If the collection exists already, this is fine.


## RAG Query & Response Generation
Query the index for context given a user-supplied question, and use that context to generate a response

**Note:** You can also run this as a CLI script using the Example command: 
```
!python rag.py query "In July, flour sold at Boston for _?" -k=10 --chroma-collection-name us_history_volume_7
```

In [9]:
def retrieve_data(collection, query, k):
    print("Querying for:", query)
    context = collection.query(query_texts=[query], n_results=k)
    return context


def serialize_retrieved_data(data):
    # print("Serializing data:", type(data), data)
    return "\n".join(f"{k}={v}" for k, v in data.items())


async def generate(query, context, query_index):
    config = AIConfigRuntime.load("rag.aiconfig.yaml")
    params = {f"query{query_index}": query, f"context{query_index}": context}
    # print("Running generate with params:", params)
    return await config.run_and_get_output_text(
        f"generate_{query_index}", params=params
    )

async def run_query(query, collection_name, k, query_index):
    chroma_client = chromadb.PersistentClient(path=chromadb_path)
    collection = chroma_client.get_collection(name=collection_name)
    data = retrieve_data(collection, query, k)
    print("Retrieved data:\n", "\n".join(data["documents"][0]))
    context = serialize_retrieved_data(data)
    result = await generate(query, context, query_index)
    print("\n\nResponse:\n", result)

    return (query, context, result)

In [10]:
queries = [
    "What was the price of flour sold in Boston in August?",
    "When and why did the british Blockade happen?"    
]

query_index = 1

In [11]:
query, context, result = await run_query(
    queries[query_index], collection_name, k=10, 
    query_index=query_index
)

Querying for: When and why did the british Blockade happen?


2024-02-08 16:38:56.540151 [W:onnxruntime:, helper.cc:67 IsInputSupported] CoreML does not support input dim > 16384. Input:embeddings.word_embeddings.weight, shape: {30522,384}
2024-02-08 16:38:56.540612 [W:onnxruntime:, coreml_execution_provider.cc:81 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 49 number of nodes in the graph: 323 number of nodes supported by CoreML: 231


Retrieved data:
 antic during the
winter months.

With it went the tale of Napoleon’s immense disaster. October 23 he
began his retreat; November 23 he succeeded in crossing the Beresina
and escaping capture; December 5 he abandoned what was still left of
his army; and December 19, after travelling secretly and without rest
across Europe, he appeared suddenly in Paris, still powerful, but in
danger. Nothing could be better calculated to support the Russian
mediation in the President’s mind. The possibility of remaining without
a friend in the world while carrying on a war without hope of success,
gave to the Czar’s friendship a value altogether new.

Other news crossed the ocean at the same time, but encouraged no hope
that England would give way. First in importance, and not to be trifled
with, was the British official announcement, dated December 26, 1812,
of the blockade of the Chesapeake and Delaware. Americans held that
this blockade was illegal,[18]--a blockade of a coast, not of

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)




Response:
 The British blockade happened in 1812. It occurred because of the British official announcement, dated December 26, 1812, of the blockade of the Chesapeake and Delaware. The Americans held that this blockade was illegal, but the British enforced it with energy, making Chesapeake Bay, Delaware River, and the Vineyard Sound little better than British waters. The blockade was a significant military measure that caused serious harm to the United States.


## Evaluate the response
Run evals on the responses across 
* **relevance** -- does the answer make sense in context of the original question?, 
* **faithfulness** -- is the final answer faithful to the data that we fed into the LLM?
* **coherence** -- is the answer consistent and easy to understand?
* **succinctness** -- does the answer contain unnecessary information?

In [12]:
async def run_evals(query, context, answer, query_index):
    config = AIConfigRuntime.load("rag.aiconfig.yaml")
    
    return [
        await config.run_and_get_output_text(
            f"evaluate_{criterion}_{query_index}",
            params={
                f"query_{query_index}": query,
                f"context_{query_index}": context,
                f"generate_{query_index}": {
                    "output": answer,
                }
            },
        )
        for criterion in [
            "relevance", "faithfulness", "coherence", "succinctness"
        ]
    ]


In [13]:
print(f"Evaluating...Query: {query} \n Answer: {result}")
evals = await run_evals(query, context, result, query_index)
print("Evaluations:")
for criterion, score in zip(
    ["relevance", "faithfulness", "coherence", "succinctness"], 
    evals
):
    print(f"{criterion}: {score}")


Evaluating...Query: When and why did the british Blockade happen? 
 Answer: The British blockade happened in 1812. It occurred because of the British official announcement, dated December 26, 1812, of the blockade of the Chesapeake and Delaware. The Americans held that this blockade was illegal, but the British enforced it with energy, making Chesapeake Bay, Delaware River, and the Vineyard Sound little better than British waters. The blockade was a significant military measure that caused serious harm to the United States.
Evaluations:
relevance: The response is missing.
faithfulness: NO

Explanation: There is no context provided to evaluate the given answer against, therefore, it is not possible to determined if the answer adheres to the context.
coherence: Yes, the answer is self-consistent and easy to understand. It gives clear information about the cause and effects of the British blockade in 1812, its legal context, and its geographical extent. The language is straightforward and

In [14]:
!python rag.py info

/opt/homebrew/Caskroom/miniconda/base/envs/rag-demo-1/lib/python3.10/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_parsers" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Starting info
Available Chroma Collections: [Collection(name=us_history_volume_7)]
